# Introduction

This is a report by Vita Naglič and Arjan Skok on the first assignment in the Intelligent Systems course. The goal of the assignment is to utilize a genetic algorithm to create expressions, that would best fit the given input-out pairs.
The assignment is done in python using the pygad package. 

# The setup:

First we have to ensure, that we are able to create valid trees. We can do that in 2 different ways:
- either using an already existing equation, which we transform into a prefix mode, using the 'infix_to_prefix()' function and then transforming it into a tree, using the 'buildTree()' function:

In [ ]:
class Node:
    def __init__(self, value):
        self.value = value
        self.left = None
        self.right = None

def buildTree(expression):
    if len(expression) > 0:  # Check if there are elements in the list
        root = Node(expression[0])
        expression.pop(0)
        if is_float(root.value) or root.value.lstrip('-+').isalpha():
            return root

        root.left = buildTree(expression)
        root.right = buildTree(expression)
    else:
        root = None
        
    return(root)

- or generating a random valid tree, using the 'generateTree()' function (simmilar to treeBuild, the difference being that we generate a random expression in each node instead of reading it from the equation)

A valid tree is  one that has the operands at its leaves and operators in central nodes.
Afterwards we need to read our input and desired output data, also known as X and Y. The dataset.csv contains 98 such pairs, and one already known equation that connects each pair. We will use this equation for validating our functions.
We pick one of the pairs for our algorithm run.

With that done, we need to generate our initial population of a determined number of trees. Because pygad can't take tree structures as initial population, we change the trees into a list of pairs [type, value], where type=0 means the value is a number, type=1 operator and type=2 a variable X.

With that done, we can focus on the genetic algorithm itself.

# Fitness function

The fitness function that we use is: 

In [ ]:
def fitness_func(ga_instance, solution, solution_idx):
    # Convert the solution array into an expression tree
    solution_tree = arrayToTree(solution)
    # Evaluate the tree with the input data
    predicted_output = [evaluateTree(solution_tree, x) for x in xs]

    # Calculate the mean squared error
    mse = np.mean((np.array(predicted_output) - np.array(desired_output))**2)

    # Return the fitness value (the reciprocal of the error)
    try:
        fitness = 1.0 / (1 + mse)
    except:
        fitnes = 0.000000001

    # If the fitness value is complex, return its real part
    if isinstance(fitness, complex) and fitness < 0.1:
        return 0.000000001
    # Otherwise, return the fitness value as is
    else:
        return fitness

The fitness function works as such:
- We transform the solution treeArray into a tree
- We calculate the predicted output of the solution tree for each X in the input data for this case
- We calculate the mean square difference between the predicted and the optimal output of the equation
- We punish the equations that are longer than the given equation from the dataset
- Fitness is calculated as 1 / (1 + mean squared error + penalty factor)

# Crossover and mutation

Next we define our custom crossover and mutation function.

### Mutation

This is the mutation function:

In [ ]:
def mutation(offspring, instance):
    for idx, arr in enumerate(offspring):
        tree = arrayToTree(arr)
        tree_count = countTree(tree)
        if tree_count > 1:
            mutation_point = random.randint(1, tree_count)
        else:
            mutation_point = 1  # or some other default value
            
        operators = ['+', '-', '*', '/', '^']
        subtree = poddrevo_gen(tree, mutation_point)
        if subtree is not None:
            if subtree.value in operators:
                subtree.value = operators[random.randint(0, len(operators) - 1)]
            else:
                operands = list(range(-10,10))
                operands.extend(['x', '-x'])
                subtree.value = str(operands[random.randint(0, len(operands) - 1)])
            offspring[idx] = treeToArray(tree, 300)
    return offspring

In a tree we select a random node in the tree, using the poddrevo_gen() function. After that we change the value of this node accordingly (operator into operator, operand into operand), so the mutation doesn't yield nonvalid expressions.

### Crossover

This is the crossover function:

In [ ]:
def crossover(parents, offspring_size, instance):
    offspring = []
    num_parents = len(parents)
    while len(offspring) < offspring_size[0]:
        # Select two parents
        arr1 = parents[random.randint(0, num_parents-1)]
        arr2 = parents[random.randint(0, num_parents-1)]
        arc1 = arr1.copy()
        arc2 = arr2.copy()
        tree1 = arrayToTree(arc1)
        tree2 = arrayToTree(arc2)
        tree_count = min(countTree(tree1), countTree(tree2))
        if tree_count > 1:
            crossover_point = random.randint(1, tree_count)
        else:
            crossover_point = 1  # or some other default value
        poddrevo1 = poddrevo_gen(tree1, crossover_point)
        poddrevo2 = poddrevo_gen(tree2, crossover_point)

        if poddrevo1 is not None and poddrevo2 is not None:
            poddrevo1.left, poddrevo2.left = poddrevo2.left, poddrevo1.left
            poddrevo1.right, poddrevo2.right = poddrevo2.right, poddrevo1.right
            poddrevo1.value, poddrevo2.value = poddrevo2.value, poddrevo1.value

        # Add the offspring to the list
        offspring.append(treeToArray(tree1, 300))
        # If there is room for another offspring, add it
        if len(offspring) < offspring_size[0]:
            offspring.append(treeToArray(tree2, 300))

    return np.array(offspring) 

We select 2 parents and a node in each tree. Using the poddrevo_gen() function, we find this node. Afterwards we swap the 2 subtrees (we swap their children and the value of the selected node).

# Onto the genetic algorithm

After all this work is done, we can finally set up the algorithm. We set up the pygad.GA instance:

In [ ]:
ga_instance = pygad.GA(num_generations=800,
                        num_parents_mating=50,
                        fitness_func=fitness_func,
                        initial_population=population,
                        parent_selection_type="tournament",
                        gene_type=np.float64,
                        mutation_type=mutation,
                        crossover_type=crossover, 
                        mutation_probability=0.08)
                    

ga_instance.run()

We extract the best solution (the one with the highest fitness) and display the graphs and print the result.

In [ ]:
solution_array, solution_fitness, solution_idx = ga_instance.best_solution()

print("\nsolution: ", solution_array)
    solution_tree = arrayToTree(solution_array)
    
    plot(fun_optimal_tree, solution_tree, xs)
    ga_instance.plot_fitness()
    print("solution fitness: ", solution_fitness)
    print("\nsolution: ")
    printTree(solution_tree)
    print("\noptimal: ")
    printTree(fun_optimal_tree)
    print("\n")

# Results and analysis

We tested the algorithm on 3 datasets with different complexity and an intial population of 100 equations . We will perform the test on each 2 times, first with 100 generations and then with 600

We can see that the results are correct, or at least close for the lower complexity equations, but with bigger one there are more problems and lower accuracy results.




| Run | Equation 1 | Equation 2 | Equation 3 |
| --- | --- | --- | --- |
| Pre-given equation | (x * 5) | ((x * 5) - 4) / 6 | -4 * x ** 5 + 1 * x ** 3 + -2 * x ** 2 + -1 * x + -1 |
| 100 generations | -5.0 * -x | (-x / 6) - (-x) | x * (9 * -x - (-x - x) * x) |
| Fitness 100 gen | 1 | 0.6923 | 3.8272e-13 |
| 600 generations | x * 5 | x + (x / -6) | (x + x) * x * x |
| Fitness 600 gen | 1 | 0.6923 | 3.8238e-13 |


### Equation 1 (100):

<div>
<img src="./slike/primer1..png" width="600" />
<img src="./slike/primer1 diff.png" width="1000" />

### Equation 1 (600):
<img src="./slike/primer2.png" width="600" /> 
<img src="./slike/primer 2 diff.png" width="1000" /> 

### Equation 2 (100):
<img src="./slike/Figure_1.png" width="600" /> 
<img src="./slike/ffff.png" width="1000" /> 

### Equation 2 (600):
<img src="./slike/f2.png" width="600" /> 
<img src="./slike/f222.png" width="1000" /> 

### Equation 3 (100):
<img src="./slike/f3.png" width="600" /> 
<img src="./slike/ff3.png" width="1000" /> 

### Equation 3 (600):
<img src="./slike/f4.png" width="600" /> 
<img src="./slike/ff4.png" width="1000" /> 

</div>
